In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm

In [3]:

model_checkpoints = "gogamza/kobart-base-v2"


tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints) 

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [4]:
train_df = pd.read_csv('data/train_total.csv')
val_df = pd.read_csv('data/val_total.csv')

In [5]:
train_Category = train_df['Category'].unique()
val_Category = val_df['Category'].unique()
def categori_ext(data, Category, tv):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
    return df

In [6]:
train_df = categori_ext(train_df, train_Category, 'train')
val_df = categori_ext(val_df, val_Category, 'val')

In [7]:
train_df.head()

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)


In [8]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)
test_samples = Dataset.from_pandas(val_df)

In [9]:
print(train_data)
print(val_data)
print(test_samples)

Dataset(features: {'Id': Value(dtype='string', id=None), 'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None), 'Category': Value(dtype='string', id=None)}, num_rows: 13994)
Dataset(features: {'Id': Value(dtype='string', id=None), 'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None), 'Category': Value(dtype='string', id=None)}, num_rows: 1746)
Dataset(features: {'Id': Value(dtype='string', id=None), 'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None), 'Category': Value(dtype='string', id=None)}, num_rows: 1746)


In [10]:
max_input = 128
max_target = 32
batch_size = 4
ignore_index = -100# tokenizer.pad_token_id

In [11]:
 def add_ignored_data(inputs, max_len, ignore_index):
        if len(inputs) < max_len:
            pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs

In [12]:
def add_padding_data(inputs, max_len):
        pad_index = tokenizer.pad_token_id
        if len(inputs) < max_len:
            pad = [pad_index] *(max_len - len(inputs))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs 

In [13]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []

    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i]), max_input))
    for i in range(len(data_to_process['Summary'])):
        label_id.append(tokenizer.encode(data_to_process['Summary'][i]))  
        label_id[i].append(tokenizer.eos_token_id)  
    for i in range(len(data_to_process['Summary'])):  
        dec_input_id = [tokenizer.eos_token_id]
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Summary'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

In [14]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
                              
model.config.max_length = 32 

In [16]:
model.config

BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "kobart_version": 2.0,
  

In [17]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)
   

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoint/non_test",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-05,
    weight_decay=0.1,
    #label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch'

    #evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?
"""
DataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . 
또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.
tokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.
"""

'\nDataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . \n또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.\ntokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.\n'

In [20]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
   # callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Category, Id. If Category, Id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 13994
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4375
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,2.697300,2.342048,"{'r': 0.19840706126435292, 'p': 0.22694401402323766, 'f': 0.20437299538491316}","{'r': 0.06991126206506626, 'p': 0.07785951310595449, 'f': 0.07078999370548786}","{'r': 0.18964592304590125, 'p': 0.21726729980841952, 'f': 0.19550008875295527}"
2,2.104800,2.261964,"{'r': 0.19502233345372225, 'p': 0.2265323612999107, 'f': 0.20192089948152125}","{'r': 0.0695220969573183, 'p': 0.08015975768986641, 'f': 0.07114365102131166}","{'r': 0.18699928827144313, 'p': 0.21759927183283218, 'f': 0.19376373059643168}"
3,1.788100,2.282877,"{'r': 0.20189621693036758, 'p': 0.22437475761473286, 'f': 0.20496601672820006}","{'r': 0.0692114463454485, 'p': 0.07547736789176328, 'f': 0.06912914292706462}","{'r': 0.192893822467952, 'p': 0.21439881675072064, 'f': 0.1958636340431713}"
4,1.566200,2.335724,"{'r': 0.20898850713873654, 'p': 0.22268402339837196, 'f': 0.20848514930057896}","{'r': 0.07432905977365419, 'p': 0.07842693888376824, 'f': 0.0732865911062758}","{'r': 0.1995986076637956, 'p': 0.21289631534875877, 'f': 0.19917819455647745}"
5,1.422800,2.374117,"{'r': 0.20655992256876984, 'p': 0.2241043053629495, 'f': 0.20757055634802182}","{'r': 0.0720739280299568, 'p': 0.07805842761045556, 'f': 0.07189540444487122}","{'r': 0.19610765107458542, 'p': 0.21325268370300976, 'f': 0.19726629868274548}"


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Category, Id. If Category, Id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1746
  Batch size = 64
Saving model checkpoint to checkpoint/non_test/checkpoint-875
Configuration saved in checkpoint/non_test/checkpoint-875/config.json
Model weights saved in checkpoint/non_test/checkpoint-875/pytorch_model.bin
tokenizer config file saved in checkpoint/non_test/checkpoint-875/tokenizer_config.json
Special tokens file saved in checkpoint/non_test/checkpoint-875/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Category, Id. If Category, Id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
*

TrainOutput(global_step=4375, training_loss=1.9158431640625, metrics={'train_runtime': 777.4933, 'train_samples_per_second': 89.994, 'train_steps_per_second': 5.627, 'total_flos': 5332907497881600.0, 'train_loss': 1.9158431640625, 'epoch': 5.0})

In [22]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Category, Id. If Category, Id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1746
  Batch size = 64


{'eval_loss': 2.2619640827178955,
 'eval_rouge-1': {'r': 0.19502233345372225,
  'p': 0.2265323612999107,
  'f': 0.20192089948152125},
 'eval_rouge-2': {'r': 0.0695220969573183,
  'p': 0.08015975768986641,
  'f': 0.07114365102131166},
 'eval_rouge-l': {'r': 0.18699928827144313,
  'p': 0.21759927183283218,
  'f': 0.19376373059643168},
 'eval_runtime': 10.4686,
 'eval_samples_per_second': 166.785,
 'eval_steps_per_second': 2.675,
 'epoch': 5.0}

In [23]:
def generate_summary(test_samples, model):

    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)

    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, 
                            attention_mask=attention_mask, 
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            eos_token_id=tokenizer.eos_token_id,)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)


loading configuration file config.json from cache at /home/jx7789/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/f9f2ec35d3c32a1ecc7a3281f9626b7ec1913fed/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072

In [24]:
#summaries_before_tuning = []
#for test_sample in tqdm(test_samples):
#    summaries_before_tuning.append(generate_summary(test_sample, model_before_tuning)[1])
#summaries_before_tuning = list(itertools.chain(*summaries_before_tuning))    
    
summaries_after_tuning=[]
for test_sample in tqdm(test_samples):
    summaries_after_tuning.append(generate_summary(test_sample, model)[1])
summaries_after_tuning = list(itertools.chain(*summaries_after_tuning))

  0%|                                                                                 | 0/1746 [00:00<?, ?it/s]/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 32 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
100%|██████████████████████████████████████████████████████████████████████| 1746/1746 [02:32<00:00, 11.47it/s]


In [25]:
rouge.get_scores(summaries_after_tuning, test_samples["Summary"], avg=True)

{'rouge-1': {'r': 0.16535943902356573,
  'p': 0.22122823332773803,
  'f': 0.18195143432015265},
 'rouge-2': {'r': 0.05618009955804081,
  'p': 0.07258321375310152,
  'f': 0.06027034023538989},
 'rouge-l': {'r': 0.15944990608462628,
  'p': 0.21419170758416758,
  'f': 0.17576656126376963}}

In [26]:
for i in range(0, len(summaries_after_tuning), 100):
    print('idx_{} '.format(i))
    #print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()  

idx_0 

Summary after 
 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다.

Target summary 
 팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.

Text ['웅', '영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서', '웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구', '내말이... 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고... 넘 시러..', '일단']
----------------------------------------------------------------------------------------------------

idx_100 

Summary after 
 한 달 한 달 동안 본사에서 근무하는 직원이 왔다고 한다.

Target summary 
 설빙의 본사 직원이 한 달에 한 번씩 오는데 한 시간 정도 이야기하고 있다.

Text ['#@소속#본사직원이왓노..', '누가.! 아진짜??? 뭔ㄴ일이래', '한달에한번씩 온대. ㅋ 근데 한시간 정도 말하는듯. 난 면접보러온줄알앗어 ㅋ', '오 몰랐네.. 무슨얘기ㅘ지']
----------------------------------------------------------------------------------------------------

idx_200 

Summary after 
 이번 주 날씨에 대해서 이야기하고 있다.

Target summary 
 이번 주 날씨가 더웠다가 비 온다는데 별로다.

Text ['근데 이번주 날씨 완전 별로더라 오늘 27도까지', '그니까..', '올라가고', '나도 티하나입고왔오', '화수목 비오고', '장마도 아니고 이게뭐얏!', '나도 가서 가디건벗을라고 더워 이제 ㅡㅡ', '후엉.......더워더워 이제 티 한장만 입어야돼', '수욜도 비오네... 우리영화보